In [3]:
!pip install -qU langchain langchain-anthropic duckduckgo-search langchain-community

In [29]:
from IPython.display import display, Markdown
display(Markdown("Related content: Here's a summary of the two pages:\n\n**Theodore Roosevelt**\n\n* Born: October 27, 1858\n* Died: January 6, 1919\n* 26th President of the United States (1901-1909)\n* Served as Vice President for 6 months under William McKinley\n* Known for his progressive policies and conservation efforts\n* Established national parks, forests, and monuments\n* Won the Nobel Peace Prize in 1906 for ending the Russo-Japanese War\n* Elected to a full term in 1904 and groomed William Howard Taft to succeed him\n\n**Theodore Roosevelt Jr.**\n\n* Born: September 13, 1887\n* Died: July 12, 1944\n* Eldest son of President Theodore Roosevelt and First Lady Edith Roosevelt\n* Educated at private academies and Harvard University\n* Served in the military during both World Wars\n* Directed troops at Utah Beach during the Normandy landings (World War II)\n* Received the Medal of Honor for his service\n* Had a successful career in business and investment banking before joining the military."))

Related content: Here's a summary of the two pages:

**Theodore Roosevelt**

* Born: October 27, 1858
* Died: January 6, 1919
* 26th President of the United States (1901-1909)
* Served as Vice President for 6 months under William McKinley
* Known for his progressive policies and conservation efforts
* Established national parks, forests, and monuments
* Won the Nobel Peace Prize in 1906 for ending the Russo-Japanese War
* Elected to a full term in 1904 and groomed William Howard Taft to succeed him

**Theodore Roosevelt Jr.**

* Born: September 13, 1887
* Died: July 12, 1944
* Eldest son of President Theodore Roosevelt and First Lady Edith Roosevelt
* Educated at private academies and Harvard University
* Served in the military during both World Wars
* Directed troops at Utah Beach during the Normandy landings (World War II)
* Received the Medal of Honor for his service
* Had a successful career in business and investment banking before joining the military.

In [ ]:
from typing import List

from langchain_core.tools import tool
from langchain_ollama import ChatOllama


@tool
def validate_user(user_id: int, addresses: List[str]) -> bool:
    """Validate user using historical addresses.

    Args:
        user_id (int): the user ID.
        addresses (List[str]): Previous addresses as a list of strings.
    """
    return True


llm = ChatOllama(
    model="llama3.1",
    temperature=0,
).bind_tools([validate_user])

result = llm.invoke(
    "Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX."
)
res = result.tool_calls

print(result, type(result))
print(res, type(res))

content='' additional_kwargs={} response_metadata={'model': 'llama3.1', 'created_at': '2024-11-10T10:55:01.537524Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'validate_user', 'arguments': {'addresses': '["123 Fake St, Boston, MA", "234 Pretend Boulevard, Houston, TX"]', 'user_id': '123'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 2388418583, 'load_duration': 32690125, 'prompt_eval_count': 225, 'prompt_eval_duration': 153761000, 'eval_count': 42, 'eval_duration': 2200683000} id='run-b73225c4-eb20-4f50-9a3d-18e5d5a165f3-0' tool_calls=[{'name': 'validate_user', 'args': {'addresses': '["123 Fake St, Boston, MA", "234 Pretend Boulevard, Houston, TX"]', 'user_id': '123'}, 'id': 'b9e96c32-bd84-44e3-8ef7-1742f575cd1e', 'type': 'tool_call'}] usage_metadata={'input_tokens': 225, 'output_tokens': 42, 'total_tokens': 267} <class 'langchain_core.messages.ai.AIMessage'>
[{'name': 'validate_user', 'args': {'addresses': '["123 Fake St, Bosto

In [5]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("who is the next president of the United States")

'Updated results from the 2024 election for the US president. Reuters live coverage of the 2024 US President, Senate, House and state governors races. "I congratulate President Trump on his election as 47th President of the United States of America, as well as Vice President-elect J.D. Vance and their families," Bush said in a statement. The White House, official residence of the president of the United States, in July 2008. The president of the United States is the head of state and head of government of the United States, [1] indirectly elected to a four-year term via the Electoral College. [2] The officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed ... Donald J. Trump, the once and now future president, capped an improbable political comeback by defeating Vice President Kamala Harris on promises to turbocharge the economy and deport undocumented ... See the latest polls, probabilities and scenarios for the 2024 p

In [14]:
@tool
def duckduckgo_search(question: str) -> str:
    """
    Ask DuckDuckGo a question.

    Args:
        question (str): The question to ask.

    Returns:
        str: a reply.
    """
    return search.invoke(question)

In [7]:
# Custom Tool for sending Slack messages using a webhook
import requests
import json
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def send_slack_message(message: str) -> str:
    """
    Sends a message to a Slack app using Slack webhook.

    Args:
        message (str): The message to send.

    Returns:
        str: A success or error message.
    """
    # webhook_url = "https://YOUR_SLACK_WEBHOOK_URL"
    # payload = {"text": message}
    # headers = {"Content-type": "application/json"}
    return "Message sent successfully!"

    # try:
    #     response = requests.post(webhook_url, data=json.dumps(payload), headers=headers)
    #     response.raise_for_status()
    #     return "Message sent successfully!"
    # except requests.exceptions.RequestException as e:
    #     return f"Error sending message: {e}"

message = "Hello from Claude!"
send_slack_message.invoke({"message": message})
# 'Message sent successfully!'

'Message sent successfully!'

In [ ]:
from dotenv import load_dotenv
# take environment variables from .env
load_dotenv()

In [ ]:
from langchain_anthropic import ChatAnthropic

# ChatAnthropic - Claude 3 Haiku
llm = ChatAnthropic(model = "claude-3-haiku-20240307")

In [24]:
# tools
tools = [duckduckgo_search, send_slack_message]

# llm with tools
llm_with_tools = llm.bind_tools(tools)

<class 'langchain_core.tools.structured.StructuredTool'> <class 'langchain_core.tools.structured.StructuredTool'>


In [19]:
messages = [
    ("system", "You are a helpful assistant."),
    ("human", "What is LangChain Tool Calling?")
]

print(llm_with_tools.invoke(messages))
#[{'name': 'duckduckgo_search',
#  'args': {'query': 'LangChain Tool Calling'},
#  'id': 'toolu_01Bfrz1Uhu84ggZd96Ae9De8'}]

messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Send '👋 Happy Birthday from Claude! 😃' message to Slack app.")
]

print(llm_with_tools.invoke(messages).tool_calls)
#[{'name': 'send_slack_message',
#  'args': {'message': '👋 Happy Birthday from Claude! 😃'},
#  'id': 'toolu_01LKxZvHpmDrdLooo1aJWqum'}]

content='' additional_kwargs={} response_metadata={'model': 'qwen2.5', 'created_at': '2024-11-16T02:50:43.712641Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'duckduckgo_search', 'arguments': {'question': 'What is LangChain Tool Calling?'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 10658826167, 'load_duration': 572147417, 'prompt_eval_count': 262, 'prompt_eval_duration': 2633000000, 'eval_count': 126, 'eval_duration': 7234000000} id='run-537a8bb9-4fb0-454e-a180-ea5a1fab6387-0' tool_calls=[{'name': 'duckduckgo_search', 'args': {'question': 'What is LangChain Tool Calling?'}, 'id': '99bb9284-3e05-4d83-ae13-cf1fc640aea9', 'type': 'tool_call'}] usage_metadata={'input_tokens': 262, 'output_tokens': 126, 'total_tokens': 388}
[{'name': 'send_slack_message', 'args': {'message': '👋 Happy Birthday from Claude! 😃'}, 'id': '95e8b72e-3a33-4670-b4e0-83d833c8ad4a', 'type': 'tool_call'}]


In [26]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Find the answer to this question: Who is the latest president of the United States?")
]

ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"duckduckgo_search": duckduckgo_search, "send_slack_message": send_slack_message}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    print(tool_output)
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages
#[('system', 'You are a helpful assistant.'),
#  ('human', "Send '👋 Happy Birthday from Claude! 😃' message to Slack app."),
#  AIMessage(content=[{'id': 'toolu_01HAV1YsmhUWK4ZRc8FJf7cL', 'input': {'message': '👋 Happy Birthday from Claude! 😃'}, 'name': 'send_slack_message', 'type': 'tool_use'}], response_metadata={'id': 'msg_01J59CnLZws1JngV9atfKwz9', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 499, 'output_tokens': 65}}, id='run-c9eb80b8-1b94-4e96-95a9-76f9f0c45c78-0', tool_calls=[{'name': 'send_slack_message', 'args': {'message': '👋 Happy Birthday from Claude! 😃'}, 'id': 'toolu_01HAV1YsmhUWK4ZRc8FJf7cL'}]),
#  ToolMessage(content='Message sent successfully!', tool_call_id='toolu_01HAV1YsmhUWK4ZRc8FJf7cL')]

The White House, official residence of the president of the United States, in July 2008. The president of the United States is the head of state and head of government of the United States, [1] indirectly elected to a four-year term via the Electoral College. [2] The officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed ... Updated results from the 2024 election for the US president. Reuters live coverage of the 2024 US President, Senate, House and state governors races. ... New York president. Democrat Kamala Harris ... Joe Biden (born November 20, 1942, Scranton, Pennsylvania, U.S.) is the 46th president of the United States (2021-25) and was the 47th vice president of the United States (2009-17) in the Democratic administration of Pres. Barack Obama. He previously represented Delaware in the U.S. Senate (1973-2009). Donald Trump was elected as President of the United States on Wednesday, Nov. 6, 2024, the Fox News 

[('system', 'You are a helpful assistant.'),
 ('human',
  'Find the answer to this question: Who is the latest president of the United States?'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5', 'created_at': '2024-11-16T05:59:36.644361Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'duckduckgo_search', 'arguments': {'question': 'Who is the latest president of the United States?'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 1865795875, 'load_duration': 27784833, 'prompt_eval_count': 272, 'prompt_eval_duration': 161000000, 'eval_count': 31, 'eval_duration': 1672000000}, id='run-99e38f82-cd9b-4954-88e3-b0d6d4c99418-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'question': 'Who is the latest president of the United States?'}, 'id': 'ce3d0a5e-469f-4f84-b9c7-9e738b7582fe', 'type': 'tool_call'}], usage_metadata={'input_tokens': 272, 'output_tokens': 31, 'total_tokens': 303}),
 ToolMessage(con

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# Prompt for creating Tool Calling Agent
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tool Calling Agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Run Agent
query = "What is LangChain Tool Calling Agent? Please send it to Slack app."
agent_executor.invoke({"input": query})